<img src="Logo_INGV_VSM.gif" width="600" align="center">

## VSM - Volcanic and Seismic source Modelling
elisa.trasatti@ingv.it
Elisa Trasatti - INGV

**VSM - Volcanic and Seismic source Modelling** is a Python code to perform inversions of geodetic data.

**Code** https://github.com/EliTras/VSM \
**License** https://github.com/EliTras/VSM/blob/main/license.lic

This Notebook is a guide to create the input file for VSM. The last cells run VSM.

Input
- data
- forward model(s)
- inversion settings

... Before to start let's decide
- working folder
- name of the input file to be written in that folder (and where results will be stored)

If it's the very first time, it is strongly suggested to create a dedicated environment. The packages needed by VSM are listed in requirements.txt (numpy, pandas, matplotlib, corner, emcee, pyshp, jupyterlab). Instructions in readme.txt

## Filename
Add the path of the working folder and name of the VSM input file is being created

In [1]:
# path of the working folder
folder_inout = '.'
filename_in = 'VSM_input.txt'

## Input data
Input data may be either displacements (m) or velocities (m/yr). If more than an input file is considered, of course the unit must be the same (all displacements or all velocities). For all datapoints, *East* and *North* are the coordinates (metric, preferred system is projected UTM).
Accepted data types are
- InSAR
- GNSS
- Leveling
- EDM
- Tilt
- Strain

### InSAR
This data refers to InSAR data, both single inferterograms or multi-temporal InSAR. Results are typycally in terms of cumulative displacements or annual velocities. The InSAR maps are usually made of thounsands of pixels so it is needed to downsample the dataset first. VSM uses downsampled InSAR datasets. Here are the details:
- up to 10 datasets from different sensors or missions
- a single datafile is referred as ` path/sarfile.txt `
- for multiple datasets just leave a blank between fullpath names ` path/sarfile1.txt path/sarfile2.txt `
- accepted formats are comma separated file (.csv), plain text (.txt), ESRI Shapefile (.shp). Ascii files may come with or without header in the first line
- format of columns
       East North Data Error LOSx LOSy LOSz
where *LOSx*, *LOSy*, *LOSz* are the x-y-z components of the versor of the Line of Sight of the sensor.

### GNSS
GNSS data are tipycally composed of mean velocity for each component of each benchmark.
- the datafile is referred to with ` path/gpsfile.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefiles (.shp). Ascii files may come with or without header in the first line
- one row for each station
- format of columns
       East North Datax Datay Dataz Errorx Errory Errorz
where *Datax* is the x-component of the displacement or velocity. Same applies to the others and to the error associated to each component.

### Leveling
Leveling data consists of differences of the elevation of benchmarks in a given time window.
- the datafile is referred to with ` path/levfile.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefiles (.shp). Ascii files may come with or without header in the first line
- format of columns
       East North Data Error
where *Data* is the different of elevation in the given time period.

### EDM
EDM (Electro-optical Distance Measuring) technique measures the difference in the horizontal position between two points. Therefore, for each row the coordinates of the starting and ending point is required. The *Data* is one single value, that is the change in distance in a given time period.
- the file is referred to with ` path/edmfile.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefiles (.shp). Ascii files may come with or without header in the first line
- format of columns
       East1 North1 East2 North2 Data Error

### Tilt
Tilt is a measure of the displacement-gradient. Tilt is the inclination of the vertical along a horizontal direction. The horizontal derivatives of the vertical deformation correspond to the East and North components of tilt. Therefore, each row contains tilt data along x and y, and the associated error. If the tiltmeter is not oriented along East or North, data should be projected accordingly. Tilt is non-dimensional and the unit for VSM is microradians (ppm).
- the file is referred to with ` path/tiltfile.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefiles (.shp). Ascii files may come with or without header in the first line
- format of columns
       East North Datax Datay Errorx Errory

### Strain
Strain is a measure of the displacement-gradient. The volumetric strain is the unit change in volume, i.e. it combines the change of the displacement in the three components. Strain is non-dimensional and the unit for VSM is microstrains (ppm).
- the file is referred to with ` path/strainfile.txt `
- accepted formats are comma separated files (.csv), plain text (.txt), ESRI Shapefiles (.shp). Ascii files may come with or without header in the first line
- format of columns
       East North Data Error


In [2]:
#Add the input files. By default all file names are 'None'. Set to 'None those unused'

# SAR filename(s)
sar_file = 'None'
#GNSS filename
gps_file = 'None'
# leveling filename
lev_file = '../../DATA/Ischia_1987_2010/obs_lev.txt'
# EDM filename
edm_file = 'None'
#tilt filename
tlt_file = 'None'
# strain filename
srn_file = 'None'

### Weights
Due to the possibility to combine different datasets, it is needed to define their weights in the inversion. Weights can be arbitrary. SAR data have a unique weight even if it can be composed of more than one dataset.


In [3]:
# SAR weight
sar_weight = 0.
# GNSS weight
gps_weight = 0.
# leveling weight
lev_weight = 1.
# EDM weight
edm_weight = 0.
# tilt weight
tlt_weight = 0.
# strain weight
srn_weight = 0.

###  Start writing the VSM input
In this step the VSM settings about input/output and data are written in the input file as plain text. Just execute this box!

In [4]:
f = open(folder_inout+'/'+filename_in, "w")
f.write(folder_inout+'\n')
f.write(sar_file+'\n'+gps_file+'\n'+lev_file+'\n'+edm_file+'\n'+tlt_file+'\n'+srn_file+'\n')
f.write(str(sar_weight)+'\n'+str(gps_weight)+'\n'+str(lev_weight)+'\n'+str(edm_weight)+'\n'+str(tlt_weight)+'\n'+str(srn_weight)+'\n');

## Forward model
VSM accepts the most common analytical forward models usually employed for geodetic data modelling. They can be also combined to up to 10 different forward models whose parameters can be constrained by the VSM inversion.
Models are
- Mogi Point Source (Mogi, 1958)
- Mogi Finite Volume (Mc Tigue, 1987)
- Penny-shaped crack (Fialko et al., 2001)
- Spheroid (Yang et al., 1988)
- Ellipsoid (Davis, 1986)
- Fault or Dyke (Okada, 1985)

Each model is characterized by specific parameters, whose range for the search should be determined. Coordinates and dimensions are in meters while intensity parameters are in accordance with the data employed. E.g., if data are displacements (meters), the volume variation is in m$^3$ while if ground velocities (m/yr) are used as input data, they lead to volume variation rate m$^3$/yr). Same applies to overpressure for pressurized sources and amount of slip for the fault model.

**Additional info**

Initially, it must be defined
- the elastic constants sch as shear modulus and Poisson ratio
- the total number of sources
- the list of sources identifiers

In [5]:
import numpy as np

# shear modulus (Pa)
mu = 5e9
# Poisson ratio
ni = 0.25

# number of sources
num_sources = 2

# write this part in the input file
f.write(str(mu)+'\n')
f.write(str(ni)+'\n')
f.write(str(num_sources)+'\n');

### Definition of forward model

Define the sources with the range of search for each parameter. For each chosen source it must be reported the list of the parameters with minimum and maximum (minimum $<=$ maximum). To fix a parameter, just choose minimum and maximum equal to the desired value. 

Each forward model is identified by a number from 0 to 5. Below are reported details on parameters and a sample of instructions to copy and paste in the code cells.

#### Mogi Point Source
**Model 0** The Mogi (1958) point source model represents an isotropic point source. Parameters:
- East coordinate of the source center
- North coordinate of the source center
- Depth of the source source center
- Volume variation

*Note* \
Depth is positive.

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`Dvol  = (0,0)` \
`bounds = [east, north, depth, Dvol] `

#### Mogi Finite Volume
**Model 1** The McTigue (1987) finite volume model is an evolution of the Mogi model and represents a spherical source with finite volume. Parameters:
- East coordinate of the source center
- North coordinate of the source center
- Depth of the source center
- Radius of the sphere
- Overpressure vs shear modulus ratio

*Note* \
Depth is positive. The overpressure vs shear modulus is adimensional.

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`radius  = (0,0)` \
`DP_mu  = (0,0)` \
`bounds = [east, north, depth, radius, DP_mu] `

#### Penny-shaped crack
**Model 2** The penny shaped crack (Fialko et al., 2001) is a disk with a radius and no vertical extension. Parameters:
- East coordinate of the disk center
- North coordinate of the disk center
- Depth of the disk center
- Radius of the disk
- Overpressure vs shear modulus ratio

*Note* \
Depth is positive. The overpressure vs shear modulus is adimensional.

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`radius  = (0,0)` \
`DP_mu  = (0,0)` \
`bounds = [east, north, depth, radius, DP_mu] `

#### Spheroid
**Model 3** The spheroid (Yang et al., 1988) is a finite-volume cavity with a constant overpressure on the boundary. It can be arbitrarily oriented in space Parameters:
- East coordinate of the source center
- North coordinate of the source center
- Depth of the source center
- Semi-major axis
- Ratio of semi-minor vs semi-major axes
- Overpressure vs shear modulus ratio (adimensional)
- Strike angle
- Dip angle

*Note* \
Depth is positive. The ratio is < 1. The overpressure vs shear modulus is adimensional. The strike is considered according to Yang et al. (1988), i.e., degrees counter clockwise from East. It means if the spheroid dips to East then strike = 0°, if it dips to North the strike = 90° etc. The dip angle is 0° = horizontal 90°= vertical.

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`axis  = (0,0)` \
`ratio  = (0,0)` \
`DP_mu  = (0,0)` \
`strike  = (0,0)` \
`dip  = (0,0)` \
`bounds = [east, north, depth, axis, ratio, DP_mu, strike, dip] `

#### Ellipsoid
**Model 4** The ellipsoid (Davis, 1986) is a point source defined by its equivalent combination of dipoles and double forces. The P$_{ij}$ matrix must be defined, whose diagonal elements are the dipoles, while the off-diagonal are the double-forces according to catersian coordinates. The results must be interpreted following Davis (1986) tables to find the shape of the ellipsoid and its orientation. Parameters:
- East coordinate of the source center
- North coordinate of the source center
- Depth of the source center
- P$_{xx}$
- P$_{yy}$
- P$_{zz}$
- P$_{xy}$
- P$_{yz}$
- P$_{zx}$

*Note* \
Depth is positive. According to Davis (1986), P$_{ij}$ matrix is expressed as Pascal multiplied by volume vs shear modulus.

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`Pxx  = (0,0)` \
`Pyy  = (0,0)` \
`Pzz  = (0,0)` \
`Pxy  = (0,0)` \
`Pyz  = (0,0)` \
`Pzx  = (0,0)` \
`bounds = [east, north, depth, Pxx, Pyy, Pzz, Pxy, Pyz, Pzx] `

#### Fault or Dike
**Model 5** The fault model (Okada, 1985) is a rectangular plane undergoing shear slip and/or tensile opening/closing. Two configurations have been employed. The first uses strike-slip and dip-slip on the fault, while the other uses the total slip and the rake angle. Parameters:
- East coordinate of the top left corner
- North coordinate of the top left corner
- Depth of the top
- Length
- Width
- Strike angle
- Dip angle
- Strike-slip, or, conversely, total slip
- Dip-slip, or, conversely, rake angle
- Tensile movement

*Note* \
Depth is positive. The strike angle is the orientation of the plane seen from the top left corner and it is measured according to Okada (1985), from North conter-clockwise, i.e., strike = 0° if the fault plane is oriented North-South and dips to East, strike = 90° if the fault is East-West trening and dips to the South, etc. The tensile movement is > 0 for opening, < 0 for closure.

*Code* \
`east = (0,0)` \
`north = (0,0)` \
`depth  = (0,0)` \
`length  = (0,0)` \
`width  = (0,0)` \
`strike  = (0,0)` \
`dip  = (0,0)` \
`param1  = (0,0)` \
`param2  = (0,0)` \
`opening  = (0,0)` \
`bounds = [east, north, depth, length, width, strike, dip, param1, param2, opening] `

In [6]:
# define the source identifier of the forward model
# 0=Mogi 1=McTigue 2=Sill 3=Spheroid 4=Ellipsoid 5=Fault
# comment or uncomment the lines needed, based on the number of sources employed

sorg_identifier = 2
#parameters minimum and maximum (minimum <= maximum)
east = (405000,409000)
north = (4506000.,  4510000.)
depth  = (500,3000)
radius  = (800,2500)
DP_mu  = (-5e-4,-1e-6)
bounds = [east, north, depth, radius, DP_mu] 
# write this part in the input file
if(sorg_identifier!= 5):
    f.write(str(sorg_identifier)+'\n')
else:
    f.write(str(sorg_identifier)+' '+okada_mode+'\n')
       
for k in range(len(bounds)):
    row = str(list(bounds[k])[0])+'\t'+str(list(bounds[k])[1])+'\n'
    f.write(row)
    
sorg_identifier = 5
#parameters minimum and maximum (minimum <= maximum)
east = (405500,405500)
north = (4510800.,4511200.)
depth  = (200,200)
length  = (800,1300)
width  = (500,500)
strike  = (90,90)
dip  = (80,80)
param1  = (0,2)
param2  = (-90,-90)
opening  = (0,0)
bounds = [east, north, depth, length, width, strike, dip, param1, param2, opening] 
okada_mode = 'R'
# write this part in the input file
if(sorg_identifier!= 5):
    f.write(str(sorg_identifier)+'\n')
else:
    f.write(str(sorg_identifier)+' '+okada_mode+'\n')
       
for k in range(len(bounds)):
    row = str(list(bounds[k])[0])+'\t'+str(list(bounds[k])[1])+'\n'
    f.write(row)

## VSM settings

### Inversion algorithms
There are two inversion tools included in VSM
- **NA** is the **Neighbourhood Algorithm**, a global optimizer based on the Voronoi cells theory for the sampling (Sambridge, 1999). The parameters needed are:
    - number of samples at each iteration `sampl1`
    - number of re-samples from each iteration `sampl2`
    - number of iterations `sampl3`
- **BO** is the **Bayesian Optimization**, a global optimizer based on Bayesian inference and MCMC sampling. The parameters needed are:
    - number of random walks `sampl1`
    - number of steps for each random walk `sampl2`

In [7]:
# optimizer choice NA = 0 BO = 1
optimizer_choice = 0
# accordingly define the sampling parameters
sampl1 = 1000
sampl2 = 300
sampl3 = 15

### Plots
It can be decided to have the plots done or not, specifically the statistics related to the 1D and 2D parameters distributions. Accordingly, also the number of burn-in (i.e., excluded) models can be defined, considering that at the beginning both the optimizers sample low probability models also. Define
- plot `yes` or `no`
- number of burn-in models

In [8]:
# plot yes or not? Accepted 'yes' 'YES' 'y' 'Y'
plotyn = 'y'
num_skip = 1000

In [9]:
# write this last part in the input file
f.write(str(optimizer_choice)+'\n')
if(optimizer_choice == 0):
    f.write(str(sampl1)+' '+str(sampl2)+'\n')
    f.write(str(sampl3)+'\n')
else:
    f.write(str(sampl1)+'\n')
    f.write(str(sampl2)+'\n')
f.write(plotyn+' '+str(num_skip)+'\n')

f.close()

## And now?
Now the VSM input is stored in the specified folder and ready to be used in a VSM run.
VSM may be launched by
- Jupyter Notebook, e.g.,  VSM_JN.ipynb
- Python code, e.g., VSM_test.py
- Graphical Unit Interface VSM_GUI.py

In [10]:
import sys
sys.path.append('../../VSM')
import VSM

In [11]:
VSM.read_VSM_settings(folder_inout+'/'+filename_in)



*******************************************************************************

                   VSM exectution begins

Start reading settings of VSM from input file:
./VSM_input.txt 

SAR data file # 1 None
GNSS data file None
Leveling data file ../../DATA/Ischia_1987_2010/obs_lev.txt
EDM data file None
Tilt data file None
Strain data file None

Source 1 considered:  Penny-shaped crack (Fialko et al., 2001)

Source 2 considered:  Fault or Dike (Okada, 1985) with mode R

There are 15 total free parameters
Parameters actually inverted: 8
xcen (405000.0, 409000.0)
ycen (4506000.0, 4510000.0)
depth (500.0, 3000.0)
radius (800.0, 2500.0)
dP_mu (-0.0005, -1e-06)
ytlc_2 (4510800.0, 4511200.0)
length_2 (800.0, 1300.0)
param1_2 (0.0, 2.0)

Optimization tool chosen --> NEIGHBOURHOOD ALGORITHM
N. of samples:  1000 N. of re-samples 300
N. of iterations:  15

Plot? y Burn-in 1000


In [ ]:
VSM.iVSM()


Reading data...
Found  146 Leveling data

Output folder is --> ./

------------ALL INPUT READ------------


NEIGHBOURHOOD ALGORITM running...
Searcher(iteration=1, samples=1000, best=6.274149e+01)
Searcher(iteration=2, samples=2000, best=3.438672e+01)
Searcher(iteration=3, samples=3000, best=3.161993e+01)
Searcher(iteration=4, samples=4000, best=2.949030e+01)
Searcher(iteration=5, samples=5000, best=2.949030e+01)
Searcher(iteration=6, samples=6000, best=2.826663e+01)
Searcher(iteration=7, samples=7000, best=2.397036e+01)
Searcher(iteration=8, samples=8000, best=2.397036e+01)
Searcher(iteration=9, samples=9000, best=2.397036e+01)
Searcher(iteration=10, samples=10000, best=2.397036e+01)
Searcher(iteration=11, samples=11000, best=2.397036e+01)
Searcher(iteration=12, samples=12000, best=2.397036e+01)
Searcher(iteration=13, samples=13000, best=2.364847e+01)
